In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import os
import sys
os.environ['PYTHONPATH'] = os.environ['PYTHONPATH'] + 'C:\\Users\\rdpuser\\Documents\\Github\\parkit;'
sys.path.append('C:\\Users\\rdpuser\\Documents\\Github\\parkit')

In [3]:
import logging
from parkit import *
logger = logging.getLogger(__name__)

In [4]:
set_default_site('C:\\Users\\rdpuser\\Desktop\\test', create = True)

In [5]:
# Collection Tests

In [6]:
get_default_site()

('C:\\Users\\rdpuser\\Desktop\\test', 'a15e6c7e-d992-47a2-bd7a-a3798eabcff8')

In [7]:
bind_symbols('module')

['collection_tests']

In [9]:
collection_tests(1000, sync = True)

True

In [10]:
disable_tasks()
compactify()
assert len([task for task in tasks()]) == 0

In [11]:
list(tasks())

[]

In [12]:
pidtable.get_snapshot()

{6208: {'create_time': 1624706315.4986243,
  'process_uid': '17ea1276-7fc4-4285-ac1d-803815365b25',
  'node_uid': None,
  'cluster_uid': None},
 11220: {'create_time': 1624706367.6278,
  'process_uid': '160a939a-5ba0-481d-a534-51d1b28618ba',
  'node_uid': None,
  'cluster_uid': None},
 9968: {'create_time': 1624706265.2370863,
  'process_uid': '795376f3-d072-4823-9e6e-cc36547161ea',
  'node_uid': None,
  'cluster_uid': None}}

In [13]:
enable_tasks()

True

In [14]:
set_concurrency(4)

In [15]:
import random

tasks = []
for i in range(16):
    tasks.append(collection_tests(1000))
while True:
    assert len(tasks) == 16
    if all(task.status in ['crashed', 'failed', 'finished'] for task in tasks):
        break
    choice = random.choice([1, 2])
    if choice == 1:
        set_concurrency(random.randint(1, 10))
    elif choice == 2:
        enable_tasks()
assert all(task.result for task in tasks)
assert all(task.status == 'finished' for task in tasks)
assert all(task.error is None for task in tasks)

In [25]:
[task.status for task in tasks]

ObjectNotFoundError: 

In [17]:
get_concurrency()

4

In [18]:
disable_tasks()

True

In [ ]:
# Scheduler Tests

In [19]:
@asyncable
def foo():
    logger.info('foo')

In [20]:
enable_tasks()

True

In [22]:
schedule(foo, frequency = Frequency.SECOND, period = 1, max_times = 10, start = 'now')

In [23]:
[s.cancel() for s in schedulers()]

[None, None]

In [24]:
compactify()

In [ ]:
# File Tests

In [26]:
f = File()

In [27]:
f.set_content('hello')

In [28]:
assert len(f.get_content()) == 5

In [29]:
assert f.get_content() == 'hello'

In [30]:
assert f.encoding == 'utf-8'

In [31]:
assert f.metadata['content-type'] == 'text/plain'

In [32]:
f.set_content(b'hello')

In [33]:
assert f.get_content() == b'hello'

In [34]:
assert isinstance(f.get_content(), bytearray)

In [35]:
with snapshot(f):
    assert isinstance(f.get_content(zero_copy = True), memoryview)

In [36]:
assert f.metadata['content-type'] == 'application/octet-stream'

In [37]:
f.set_content([1, 2, 3])

In [38]:
assert f.get_content() == [1, 2, 3]

In [39]:
assert f.metadata['content-type'] == 'application/python-pickle'

In [40]:
import numpy as np
import pandas as pd

In [41]:
rows = []
for i in range(1000000):
    rows.append(dict(a = i, b = i))
df = pd.DataFrame(rows)

In [42]:
f = File(mode = 'wb')

In [43]:
assert f.size == 0

In [44]:
f.set_content(df)

In [45]:
assert df.equals(f.get_content())

In [46]:
a = np.random.rand(100,100)

In [50]:
f = File('test', mode = 'wb+', create = True)
with f:
    np.save(f, a)

In [51]:
f.mode = 'rb+'
with f:
    assert np.array_equal(a, np.load(f))

In [ ]:
f = File('test', mode = 'w+')
with f:
    np.savetxt(f, a)

In [ ]:
f.mode = 'r+'
with f:
    assert np.array_equal(a, np.loadtxt(f))

In [48]:
for obj in directory('__task__', include_hidden = True):
    print(obj.path)

__task__/__cluster_state_dict__
__task__/__node_termination_queue__
__task__/__submit_queue__
